In [1]:
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [3]:
learning_rate = 1e-3
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = datasets.MNIST(root='D:/Dataset/MNIST_data/',
                                                 train=True,
                                                 transform=transforms.ToTensor(),
                                                 download=True)

mnist_test = datasets.MNIST(root='D:/Dataset/MNIST_data/',
                                                 train=False,
                                                 transform=transforms.ToTensor(),
                                                 download=True)

C:\anaconda\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
data_loader = DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # first layer
        # Image shape = (?, 28, 28, 1)
        #   Conv   -> (?, 28, 28, 32)
        #   Pool   -> (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # second layer
        # Image shape = (?, 14, 14, 32)
        #   Conv   -> (?, 14, 14, 64)
        #   Pool   -> (?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = nn.Linear(7 * 7 * 64, 10, bias=True) # Full Connected Layers
        
        nn.init.xavier_uniform_(self.fc.weight) # Only Full Connected Layers Reset Weight
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # Flatten
        out = self.fc(out)
        return out

In [7]:
model = CNN().to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_batch = len(data_loader)
print('Total number of batches : {}'.format(total_batch))

Total number of batches : 600


In [10]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\anaconda\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch:    1] cost = 0.218759373
[Epoch:    2] cost = 0.0596405379
[Epoch:    3] cost = 0.044724796
[Epoch:    4] cost = 0.0344746411
[Epoch:    5] cost = 0.0296490639
[Epoch:    6] cost = 0.024573883
[Epoch:    7] cost = 0.0204745419
[Epoch:    8] cost = 0.017667897
[Epoch:    9] cost = 0.0150095653
[Epoch:   10] cost = 0.0125000346
[Epoch:   11] cost = 0.0100905579
[Epoch:   12] cost = 0.00961322337
[Epoch:   13] cost = 0.00795730203
[Epoch:   14] cost = 0.0077612591
[Epoch:   15] cost = 0.00566769531


In [11]:
with torch.no_grad():
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    
    prediction = model(X_test)
    corret_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = corret_prediction.float().mean()
    print('Accuracy :', accuracy.item())

Accuracy : 0.9872999787330627
